In [ ]:
from __future__ import print_function
import os
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
from labellines import labelLine, labelLines
from custom_model.predictor import *
from custom_model.losses import *
from custom_model.datagen import *
from custom_model.inference import *
from custom_model.inference_hist import EnsemblePrediction, Calibration
import numpy as np
import keras
from keras.optimizers import Adam
import os
from scipy.stats import beta
from scipy.stats import moment

from config import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.get_logger().setLevel('INFO')

import warnings
warnings.filterwarnings("ignore")
import time

In [ ]:
para['normalize'] = 0 # DO NOT normalize
testin_gen = DataGenerator(para, 8, 'testin') # test-2019
testout_gen = DataGenerator(para, 8, 'testout') # test-2022

# get ground-truth of both test sets
x1 = testin_gen.x
y1 = testin_gen.y

x2 = testout_gen.x
y2 = testout_gen.y

# get parameters (a, b) of beta distributions predicted by the deep ensembles
data1 = np.load('./result/2019.npz')
a1 = data1['A']
b1 = data1['B']

data2 = np.load('./result/2022.npz')
a2 = data2['A']
b2 = data2['B']

# calculate mean and variance
mu1, var1 = beta.stats(a1, b1, scale=130, moments='mv')
mu2, var2 = beta.stats(a2, b2, scale=130, moments='mv')

# calculate two types of uncertainty measure by variance
yp1 = np.mean(mu1, 0)
alea1 = np.mean(var1, 0)
epis1 = np.var(mu1, 0)

yp2 = np.mean(mu2, 0)
alea2 = np.mean(var2, 0)
epis2 = np.var(mu2, 0)

# calculate two types of uncertainty measure by entropy
Ua1, Ue1 = EntropyUncertainty(a1, b1)
Ua2, Ue2 = EntropyUncertainty(a2, b2)

In [ ]:
# plot figure 6 in the manuscript
Ua = np.zeros(10)
Ue = np.zeros(10)
Ut = np.zeros(10)
for i in range(10):
    Ua[i] = np.mean(alea2[:,i])**0.5
    Ue[i] = np.mean(epis2[:,i])**0.5
    Ut[i] = np.mean(epis2[:,i]+alea2[:,i])**0.5
#     Ua[i] = np.mean(Ea1[:,i])
#     Ue[i] = np.mean(Ee1[:,i])
#     Ut[i] = np.mean(Ea1[:,i]+Ee1[:,i])
plt.plot(np.arange(4,41,4), Ua, marker='+', label='aleatoric')
plt.plot(np.arange(4,41,4), Ue, marker='x', label='epistemic')
plt.plot(np.arange(4,41,4), Ut, marker='*', label='total', color='black')
plt.xlabel('prediction horizon (min)', fontsize=14)
plt.ylabel('entropy (nats)', fontsize=14)
plt.xticks(np.arange(4,41,4))
plt.xlim(0,44)
#plt.ylim(-3,0.5)
plt.grid(linestyle='-', color='lightgray')
plt.legend(loc='best', fontsize=13)
plt.title('Uncertainty by variance(2022)', fontsize=14)
#plt.savefig('imgs/var_un2019.png', dpi=800)
plt.show()

In [ ]:
# figure 7
fig, axes = plt.subplots(2,2,figsize=(10,10))
#fig.tight_layout()
axes[0,0].hist(np.mean(alea1,(1,2))**0.5, bins=50, alpha=0.3, density=True, label='2019', color='blue', edgecolor='grey')
axes[0,0].hist(np.mean(alea2,(1,2))**0.5, bins=50, alpha=0.3, density=True, label='2022', color='red', edgecolor='grey')
axes[0,0].set_xlim(0,20)
axes[0,0].set_xlabel('$\sigma$ (km/h)', fontsize=12)
axes[0,0].set_ylabel('probability density', fontsize=12)
axes[0,0].set_title('aleatoric (variance)', fontsize=12)

axes[1,0].hist(np.mean(Ua1,(1,2)), bins=50, alpha=0.3, density=True, label='2019',color='blue', edgecolor='grey')
axes[1,0].hist(np.mean(Ua2,(1,2)), bins=50, alpha=0.3, density=True, label='2022',color='red', edgecolor='grey')
axes[1,0].set_xlim(-3.0,-0.5)
axes[1,0].set_xlabel('Entropy (nats)', fontsize=12)
axes[1,0].set_ylabel('probability density', fontsize=12)
axes[1,0].set_title('aleatoric (entropy)', fontsize=12)

axes[0,1].hist(np.mean(epis1,(1,2))**0.5, bins=50, alpha=0.3, density=True, label='2019',color='blue', edgecolor='grey')
axes[0,1].hist(np.mean(epis2,(1,2))**0.5, bins=50, alpha=0.3, density=True, label='2022',color='red', edgecolor='grey')
axes[0,1].set_xlim(0,10)
axes[0,1].set_xlabel('$\sigma$ (km/h)', fontsize=12)
axes[0,1].set_title('epistemic (variance)', fontsize=12)

axes[1,1].hist(np.mean(Ue1,(1,2)), bins=50, alpha=0.3, density=True, label='2019',color='blue', edgecolor='grey')
axes[1,1].hist(np.mean(Ue2,(1,2)), bins=50, alpha=0.3, density=True, label='2022',color='red', edgecolor='grey')
axes[1,1].set_xlim(0.05,0.45)
axes[1,1].set_xlabel('Entropy (nats)', fontsize=12)
axes[1,1].set_title('epistemic (entropy)', fontsize=12)
axes[0,1].legend()

fig.subplots_adjust(hspace=0.3)

plt.savefig('imgs/metrics.jpg', dpi=800)
plt.show()

In [ ]:
# figure 8
loc1=1
loc2=50
loc3=111
fig, axes = plt.subplots(1, 3,figsize=(10,4))
axes[0].scatter(yp1[:,4, loc1], alea1[:,4, loc1]**0.5, s=0.2, color='red')
axes[0].set_xlim(20,120)
axes[0].set_ylim(0,35)
axes[0].set_ylabel('aleatoric $\sigma$ (km/h)', fontsize=12)
axes[0].set_xlabel('speed (km/h)', fontsize=12)
axes[0].set_title('location-1', fontsize=12)

axes[1].scatter(yp1[:,4, loc2], alea1[:,4, loc2]**0.5, s=0.2, color='blue')
axes[1].set_xlim(20,120)
axes[1].set_ylim(0,35)
axes[1].set_xlabel('speed (km/h)', fontsize=12)
axes[1].set_title('location-50', fontsize=12)

axes[2].scatter(yp1[:,4, loc3], alea1[:,4, loc3]**0.5, s=0.2, color='orange')
axes[2].set_xlim(20,120)
axes[2].set_ylim(0,35)
axes[2].set_xlabel('speed (km/h)', fontsize=12)
axes[2].set_title('location-111', fontsize=12)

plt.savefig('imgs/ushape.pdf', dpi=800)
plt.show()